<a href="https://colab.research.google.com/github/wahid028/CryptoCurrency-Price-prediction_Kaggle/blob/main/LGBM_%2B_XGB_%2B_Catboost_g_research_crypto_forecasting_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Connecting colab with kaggle using kaggle API**

In [1]:
#install kaggle
!pip install -q kaggle

In [2]:
#upload the kaggle.json file
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"learn2know","key":"ef7c1f3683ce0590f0cee38b88d74d21"}'}

In [3]:
#create a kaggle directory
!mkdir ~/.kaggle

In [4]:
#copy the kaggle.json to kaggle directory
!cp kaggle.json ~/.kaggle/

In [5]:
#permission for the json to act
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
#list all the available dataset in kaggle
!kaggle datasets list

ref                                                                   title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
muratkokludataset/date-fruit-datasets                                 Date Fruit Datasets                              408KB  2022-04-03 09:25:39           9949       1346  0.9375           
victorsoeiro/netflix-tv-shows-and-movies                              Netflix TV Shows and Movies                        2MB  2022-05-15 00:01:23           2048         72  1.0              
mdmahmudulhasansuzan/students-adaptability-level-in-online-education  Students Adaptability Level in Online Education    6KB  2022-04-16 04:46:28           6628        168  1.0              
muratkokludataset/acoustic-extinguisher-fire-

## **Get g-research-crypto-forecasting dataset**

In [7]:
!kaggle competitions download -c g-research-crypto-forecasting

100% 1.13G/1.13G [00:13<00:00, 135MB/s]
100% 1.13G/1.13G [00:13<00:00, 90.2MB/s]


In [8]:
#unzip the target file
!unzip g-research-crypto-forecasting.zip

Archive:  g-research-crypto-forecasting.zip
  inflating: asset_details.csv       
  inflating: example_sample_submission.csv  
  inflating: example_test.csv        
  inflating: gresearch_crypto/__init__.py  
  inflating: gresearch_crypto/competition.cpython-37m-x86_64-linux-gnu.so  
  inflating: supplemental_train.csv  
  inflating: train.csv               


## **LGBM + XGB + Catboost**

In [10]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Different Regressor libraries 
from lightgbm import LGBMRegressor # LGBM Regressor
import xgboost as xgb # XGB Regressor
from catboost import CatBoostRegressor # CatBoost Resgressor
import gresearch_crypto
import traceback

In [12]:
# Read the Crypto dataset files
df_train = pd.read_csv('../content/train.csv')
df_asset = pd.read_csv('../content/asset_details.csv').sort_values("Asset_ID")

In [13]:
# Modify the training dataset
df_train.replace([np.inf, -np.inf], np.nan, inplace=True)
df_train = df_train.dropna()

# Modify the Assest Dataset
df_asset.replace([np.inf, -np.inf], np.nan, inplace=True)
df_asset = df_asset.dropna()

In [14]:
# Asset dataset details are following.
df_asset

,Asset_ID,Weight,Asset_Name
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
0,2,2.397895,Bitcoin Cash
10,3,4.406719,Cardano
13,4,3.555348,Dogecoin
3,5,1.386294,EOS.IO
5,6,5.894403,Ethereum
4,7,2.079442,Ethereum Classic
11,8,1.098612,IOTA
6,9,2.397895,Litecoin


**Utility Function**
These function are used to create the some new features which are useful for the model. Special thanks to @DATAISTA0 (JULIÁN PELLER) [notebook](https://www.kaggle.com/julian3833/g-research-starter-lgbm-pipeline) for functionality.

In [15]:
# Two new features from the competition tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

# Extract Asset features
def get_features(df):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_feat['Upper_Shadow'] = upper_shadow(df_feat)
    df_feat['Lower_Shadow'] = lower_shadow(df_feat)
    return df_feat

# Extract features from dataset
def get_Xy_features(df_train, df_asset_id):
    # Modfify with respect to 'asset ID'
    df = df_train[df_train["Asset_ID"] == df_asset_id]
    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")
    
    # X, y features from the dataset.
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]
    
    return X, y

**Implementation: LGBM Regressor**

In [16]:
def LGBMRegressor_model(X, y):
        
    # LGBM Regresor Model 
    model = LGBMRegressor(n_estimators=40) # 40000
    model.fit(X, y)
    
    return model

In [17]:
# Calling the implemenation of LGBM Regressor
model_lgb = {}

for asset_id, asset_name in zip(df_asset['Asset_ID'], df_asset['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    X, y = get_Xy_features(df_train, asset_id)
    model = LGBMRegressor_model(X, y)
    
    model_lgb[asset_id] = model

Training model for Binance Coin     (ID=0 )
Training model for Bitcoin          (ID=1 )
Training model for Bitcoin Cash     (ID=2 )
Training model for Cardano          (ID=3 )
Training model for Dogecoin         (ID=4 )
Training model for EOS.IO           (ID=5 )
Training model for Ethereum         (ID=6 )
Training model for Ethereum Classic (ID=7 )
Training model for IOTA             (ID=8 )
Training model for Litecoin         (ID=9 )
Training model for Maker            (ID=10)
Training model for Monero           (ID=11)
Training model for Stellar          (ID=12)
Training model for TRON             (ID=13)


In [18]:
# Checking LGBM Regressor Model interface
x = get_features(df_train.iloc[1])
y_pred_lgb = model_lgb[0].predict([x])
y_pred_lgb[0]

1.62005851685996e-06

In [19]:
# Delete unused dataset
del x
del y_pred_lgb

**Implementation: XGB Regressor**

In [20]:
# LGBM Regressor Model Function
def XGBRegressor_model(X, y):
    
    model = xgb.XGBRegressor(
    n_estimators=50, #500
    learning_rate=0.05,
    max_depth=12,
    subsample=0.9,
    colsample_bytree=0.7,
    missing=-999,
    random_state=1111,
    tree_method='hist'  
    )
    
    model.fit(X, y)
    
    return model

In [21]:
# Calling the implemenation of XGB Regressor
model_xgb = {}

for asset_id, asset_name in zip(df_asset['Asset_ID'], df_asset['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    X, y = get_Xy_features(df_train, asset_id)
    model = XGBRegressor_model(X, y)
    
    model_xgb[asset_id] = model

Training model for Binance Coin     (ID=0 )
[07:44:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Training model for Bitcoin          (ID=1 )
[07:44:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Training model for Bitcoin Cash     (ID=2 )
[07:44:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Training model for Cardano          (ID=3 )
[07:45:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Training model for Dogecoin         (ID=4 )
[07:45:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Training model for EOS.IO           (ID=5 )
[07:45:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in f

In [22]:
# Checking XGB Regressor Model interface
x = get_features(df_train.iloc[1])
y_pred_xgb = model_xgb[0].predict(pd.DataFrame([x]))
y_pred_xgb[0]

0.03848809

In [23]:
# Delete unused dataset
del x
del y_pred_xgb

**Implementation: CatBoost Regressor**

In [24]:
# Features for CatBoost Regressor
def get_features_catboost(df,row=False):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', "timestamp"]].copy()
    df_feat['Upper_Shadow'] = upper_shadow(df_feat)
    df_feat['Lower_Shadow'] = lower_shadow(df_feat)
    
    ## Add some more feats
    df_feat["high_div_low"] = df_feat["High"]/df_feat["Low"]
    df_feat["open_sub_close"] = df_feat["Open"]-df_feat["Close"]

    ## possible seasonality, datetime  features (unlikely to me meaningful, given very short time-frames)
    times = pd.to_datetime(df_feat["timestamp"],unit="s",infer_datetime_format=True)
    if row:
        df_feat["hour"] = times.hour
        df_feat["dayofweek"] = times.dayofweek 
        df_feat["day"] = times.day 
    else:
        df_feat["hour"] = times.dt.hour
        df_feat["dayofweek"] = times.dt.dayofweek 
        df_feat["day"] = times.dt.day 
    df_feat = df_feat.drop(columns = "timestamp")
    return df_feat

# X, y Features from dataset
def get_Xy_and_Catboost(df_train, asset_id):
    df = df_train[df_train["Asset_ID"] == asset_id]
    
    df_proc = get_features_catboost(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")
    
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]

    return X, y

In [25]:
#Implementation of CatBoost Regressor
def CatBoostRegressor_model(X, y):
    model = CatBoostRegressor(iterations=20, learning_rate=0.05, depth=10, random_seed=42) #1000
    model.fit(X, y)

    return model

In [26]:
# Calling of CatBoost Regressor Model
model_cat = {}

for asset_id, asset_name in zip(df_asset['Asset_ID'], df_asset['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    X, y = get_Xy_and_Catboost(df_train, asset_id)
    model = CatBoostRegressor_model(X, y)
    
    model_cat[asset_id] = model

Training model for Binance Coin     (ID=0 )
0:	learn: 0.0054495	total: 773ms	remaining: 14.7s
1:	learn: 0.0054470	total: 1.44s	remaining: 12.9s
2:	learn: 0.0054452	total: 2.11s	remaining: 12s
3:	learn: 0.0054435	total: 2.79s	remaining: 11.1s
4:	learn: 0.0054421	total: 3.44s	remaining: 10.3s
5:	learn: 0.0054395	total: 4.12s	remaining: 9.62s
6:	learn: 0.0054369	total: 4.8s	remaining: 8.91s
7:	learn: 0.0054352	total: 5.49s	remaining: 8.23s
8:	learn: 0.0054333	total: 6.16s	remaining: 7.53s
9:	learn: 0.0054317	total: 6.84s	remaining: 6.84s
10:	learn: 0.0054303	total: 7.5s	remaining: 6.14s
11:	learn: 0.0054281	total: 8.18s	remaining: 5.45s
12:	learn: 0.0054265	total: 8.85s	remaining: 4.76s
13:	learn: 0.0054253	total: 9.51s	remaining: 4.08s
14:	learn: 0.0054242	total: 10.2s	remaining: 3.39s
15:	learn: 0.0054235	total: 10.8s	remaining: 2.71s
16:	learn: 0.0054222	total: 11.5s	remaining: 2.03s
17:	learn: 0.0054207	total: 12.2s	remaining: 1.35s
18:	learn: 0.0054195	total: 12.9s	remaining: 677ms
1

In [27]:
# Check the CatBoost model interface
x = get_features_catboost(df_train.iloc[1],row=True)
y_pred_cat = model_cat[0].predict([x])
y_pred_cat[0]

9.656070557311392e-05

In [28]:
# Delete unused dataset
del x
del y_pred_cat

**Final Prediction: LGBM + XGB + CatBoost**

In [29]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    df_pred_lgbr = df_pred
    df_pred_xgbr = df_pred
    df_pred_cbtr = df_pred
    for j , row in df_test.iterrows():
        
        # LGBM Regressor
        model_lgbr = model_lgb[row['Asset_ID']]
        x_test_lgbr = get_features(row)
        y_pred_lgbr = model_lgbr.predict([x_test_lgbr])[0]
        
        df_pred_lgbr.loc[df_pred_lgbr['row_id'] == row['row_id'], 'Target'] = y_pred_lgbr
        
        # XGB Regressor
        model_xgbr = model_xgb[row['Asset_ID']]
        x_test_xgbr = get_features(row)
        y_pred_xgbr = model_xgbr.predict(pd.DataFrame([x_test_xgbr]))[0]
        
        df_pred_xgbr.loc[df_pred_xgbr['row_id'] == row['row_id'], 'Target'] = y_pred_xgbr
        
        # CatBoost Regressor
        model_cbtr = model_cat[row['Asset_ID']]
        x_test_cbtr = get_features_catboost(row, row=True)
        y_pred_cbtr = model_cbtr.predict([x_test_cbtr])[0]
        
        df_pred_cbtr.loc[df_pred_cbtr['row_id'] == row['row_id'], 'Target'] = y_pred_cbtr

    # Display the first prediction dataframe
    if i == 0:
        pred_final = 0.5 * df_pred_lgbr + 0.2 * df_pred_xgbr +  0.3 * df_pred_cbtr
        display(pred_final)

    # Send submissions
    env.predict(pred_final)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


,row_id,Target
0,0.0,-0.000379
1,1.0,-0.000188
2,2.0,-0.000102
3,3.0,-0.000127
4,4.0,0.000269
5,5.0,0.000507
6,6.0,0.000190
7,7.0,-0.000122
8,8.0,0.000151
9,9.0,0.000398


In the Final prediction there are some computational error.